In [1]:
import pandas as pd
from FeatureDataClass import IndustryData

# Fetch our primary Resume -> Industry dataset
ind = IndustryData().preprocess_data()

# Get raw counts
industry_counts = ind['Industry'].value_counts()
print("Raw counts:")
print(industry_counts)

# Get percentage distribution
industry_percentages = ind['Industry'].value_counts(normalize=True) * 100
print("\nPercentage distribution:")
print(industry_percentages.round(2))
ind

Raw counts:
Industry
Education                    410
Mechanical Engineer          384
Electrical Engineering       384
Consultant                   368
Civil Engineer               364
Sales                        364
Management                   361
Human Resources              360
Digital Media                358
Accountant                   350
Java Developer               348
Building and Construction    345
Operations Manager           345
Architecture                 344
Testing                      344
Business Analyst             340
Aviation                     340
Finance                      339
SQL Developer                338
Public Relations             337
Health and Fitness           332
Arts                         332
Network Security Engineer    330
DotNet Developer             329
Apparel                      320
Banking                      314
Automobile                   313
Web Designing                309
SAP Developer                304
Data Science          

,Industry,resume_text
0,Accountant,education omba executive leadership university...
1,Accountant,howard gerrard accountant deyjobcom birmingham...
2,Accountant,kevin frank senior accountant inforesumekraftc...
3,Accountant,place birth nationality olivia ogilvy accounta...
4,Accountant,stephen greet cpa senior accountant 9 year exp...
...,...,...
13384,Web Designing,jessica claire montgomery street san francisco...
13385,Web Designing,jessica claire montgomery street san francisco...
13386,Web Designing,summary jessica claire 100 montgomery st 10th ...
13387,Web Designing,jessica claire montgomery street san francisco...


It seems like there are a lot of IT related jobs which aren't labelled IT.

(Information Technology)
React Developer
Python Developer
DevOps
ETL Developer
Web Desiging
DotNet Developer
Network Security Engineer
SQL Developer
Java Developer

We could potentially group these into distinct classes, as it could be hard for the agent to pick up on these niche industry domains (which are moreso job titles as opposed to industries).


I'm proposing we perform the following splits

Under Information Technology, we will now include;
DevOps
SAP Developer
Database
SQL Developer
ETL Developer
Web Desiging

Creating a new class, Software Engineering, we will now include;
Java Developer
DotNet Developer
Network Security Engineer
React Developer
Python Developer

Creating a new class


In [2]:
from categorize_industries import categorize_industries
categorize_industries(industry_counts)

```python
'Engineering': ['Mechanical Engineer', 'Electrical Engineering', 'Civil Engineer', 'Network Security Engineer'],
'Consulting': ['Consultant', 'Business Analyst'],
'Sales and Marketing': ['Sales', 'Digital Media', 'Public Relations'],
'Management': ['Management', 'Operations Manager', 'PMO'],
'Human Resources': ['Human Resources'],
'Finance': ['Accountant', 'Finance', 'Banking'],
'Software Development': ['Java Developer', 'DotNet Developer', 'SQL Developer', 'SAP Developer', 'Python Developer', 'React Developer'],
'Construction': ['Building and Construction', 'Architecture'],
'Testing and QA': ['Testing'],
'Aviation': ['Aviation'],
'Healthcare and Wellness': ['Health and Fitness'],
'Arts and Media': ['Arts', 'Designing'],
'Apparel and Fashion': ['Apparel'],
'Automotive': ['Automobile'],
'Web and Digital': ['Web Designing', 'Database', 'Information Technology', 'Blockchain'],
'Food Services': ['Food and Beverages'],
'Legal': ['Advocate'],
'Data and Analysis': ['Data Science', '

In [ ]:
from categorize_industries import update_industry_column

# Assuming your DataFrame 'ind' is already defined
# Call the function to update the 'Industry' column
ind = update_industry_column(ind)

# Check the updated distribution
print("Updated Industry Distribution:")
print(ind['Industry'].value_counts())

# Print the percentage distribution
print("\nPercentages:")
print(ind['Industry'].value_counts(normalize=True) * 100)

In [3]:
it_roles = ['DevOps', 'SQL Developer', 'ETL Developer', 'Web Designing', 'Database', 'SAP Developer']
software_eng_roles = ['Java Developer', 'DotNet Developer', 'Network Security Engineer', 'React Developer', 'Python Developer']
engineering_roles = ['Mechanical Engineer', 'Electrical Engineering', 'Civil Engineer']
business_roles = ['Consultant', 'Sales', 'Management', 'Operations Manager', 'Human Resources', 'Business Analyst', 'PMO']
media_roles = ['Digital Media', 'Public Relations', 'Arts']
finance_roles = ['Accountant', 'Finance', 'Banking']
construction_roles = ['Building and Construction', 'Architecture']
testing_roles = ['Testing']
aviation_roles = ['Aviation']
health_roles = ['Health and Fitness']
apparel_roles = ['Apparel']
automobile_roles = ['Automobile']
agriculture_roles = ['Agriculture']
legal_roles = ['Advocate']
technology_roles = ['Information Technology', 'Network Security Engineer', 'Database', 'Blockchain']
design_roles = ['Designing']
bpo_roles = ['BPO']
food_roles = ['Food and Beverages']
education_roles = ['Education']

# Update the Industry column based on role mappings
ind.loc[ind['Industry'].isin(it_roles), 'Industry'] = 'Information Technology'
ind.loc[ind['Industry'].isin(software_eng_roles), 'Industry'] = 'Software Engineering'
ind.loc[ind['Industry'].isin(engineering_roles), 'Industry'] = 'Engineering'
ind.loc[ind['Industry'].isin(business_roles), 'Industry'] = 'Business'
ind.loc[ind['Industry'].isin(media_roles), 'Industry'] = 'Media'
ind.loc[ind['Industry'].isin(finance_roles), 'Industry'] = 'Finance'
ind.loc[ind['Industry'].isin(construction_roles), 'Industry'] = 'Construction'
ind.loc[ind['Industry'].isin(testing_roles), 'Industry'] = 'Testing'
ind.loc[ind['Industry'].isin(aviation_roles), 'Industry'] = 'Aviation'
ind.loc[ind['Industry'].isin(health_roles), 'Industry'] = 'Health'
ind.loc[ind['Industry'].isin(apparel_roles), 'Industry'] = 'Apparel'
ind.loc[ind['Industry'].isin(automobile_roles), 'Industry'] = 'Automobile'
ind.loc[ind['Industry'].isin(agriculture_roles), 'Industry'] = 'Agriculture'
ind.loc[ind['Industry'].isin(legal_roles), 'Industry'] = 'Legal'
ind.loc[ind['Industry'].isin(technology_roles), 'Industry'] = 'Technology'
ind.loc[ind['Industry'].isin(design_roles), 'Industry'] = 'Design'
ind.loc[ind['Industry'].isin(bpo_roles), 'Industry'] = 'BPO'
ind.loc[ind['Industry'].isin(food_roles), 'Industry'] = 'Food'
ind.loc[ind['Industry'].isin(education_roles), 'Industry'] = 'Education'

# Check the updated distribution
print("Updated Industry Distribution:")
print(ind['Industry'].value_counts())

# Print the percentage distribution
print("\nPercentages:")
print(ind['Industry'].value_counts(normalize=True) * 100)

Updated Industry Distribution:
Industry
Business                2424
Technology              2112
Software Engineering    1437
Engineering             1132
Media                   1027
Finance                 1003
Construction             689
Education                410
Testing                  344
Aviation                 340
Health                   332
Apparel                  320
Automobile               313
Data Science             299
Agriculture              293
Legal                    291
Design                   258
BPO                      203
Food                     162
Name: count, dtype: int64

Percentages:
Industry
Business                18.104414
Technology              15.774143
Software Engineering    10.732691
Engineering              8.454702
Media                    7.670476
Finance                  7.491224
Construction             5.146015
Education                3.062215
Testing                  2.569273
Aviation                 2.539398
Health              

In [2]:
# Define the mappings
it_roles = ['DevOps', 'SQL Developer', 'ETL Developer', 'Web Designing', 'Database', 'SAP Developer']
software_eng_roles = ['Java Developer', 'DotNet Developer', 'Network Security Engineer', 
                     'React Developer', 'Python Developer']

# Update the Industry column
ind.loc[ind['Industry'].isin(it_roles), 'Industry'] = 'Information Technology'
ind.loc[ind['Industry'].isin(software_eng_roles), 'Industry'] = 'Software Engineering'

# Check the updated distribution
print("Updated Industry Distribution:")
print(ind['Industry'].value_counts())
print("\nPercentages:")
print(ind['Industry'].value_counts(normalize=True) * 100)


Updated Industry Distribution:
Industry
Information Technology       2065
Software Engineering         1437
Education                     410
Mechanical Engineer           384
Electrical Engineering        384
Consultant                    368
Civil Engineer                364
Sales                         364
Management                    361
Human Resources               360
Digital Media                 358
Accountant                    350
Operations Manager            345
Building and Construction     345
Testing                       344
Architecture                  344
Aviation                      340
Business Analyst              340
Finance                       339
Public Relations              337
Health and Fitness            332
Arts                          332
Apparel                       320
Banking                       314
Automobile                    313
Data Science                  299
Agriculture                   293
Advocate                      291
PMO     

In [3]:
# Get all rows that aren't IT or Software Engineering
other_rows = ind[~ind['Industry'].isin(['Information Technology', 'Software Engineering'])]

# Randomly sample rows from each IT and Software Engineering
it_rows = ind[ind['Industry'] == 'Information Technology'].sample(n=650, random_state=42)
sw_rows = ind[ind['Industry'] == 'Software Engineering'].sample(n=600, random_state=42)

# Combine all the dataframes
df_balanced = pd.concat([other_rows, it_rows, sw_rows])

# Shuffle the final dataframe
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Check the new distribution
print("New Industry Distribution:")
print(df_balanced['Industry'].value_counts())
print("\nPercentages:")
print(df_balanced['Industry'].value_counts(normalize=True) * 100)

New Industry Distribution:
Industry
Information Technology       650
Software Engineering         600
Education                    410
Mechanical Engineer          384
Electrical Engineering       384
Consultant                   368
Civil Engineer               364
Sales                        364
Management                   361
Human Resources              360
Digital Media                358
Accountant                   350
Building and Construction    345
Operations Manager           345
Testing                      344
Architecture                 344
Business Analyst             340
Aviation                     340
Finance                      339
Public Relations             337
Arts                         332
Health and Fitness           332
Apparel                      320
Banking                      314
Automobile                   313
Data Science                 299
Agriculture                  293
Advocate                     291
PMO                          286
Designi

In [4]:
# Shuffle the rows so we don't have Industries clustered together
df_balanced = df_balanced.sample(frac=1, random_state=42) 
df_balanced

,Industry,resume_text
9359,BPO,links jessica claire montgomery street san fra...
483,Human Resources,jessica claire 9 resumesampleexamplecom 555 43...
8412,Consultant,roger kuo 201 sansome st 202 san francisco ca ...
8188,Health and Fitness,summary skills jessica claire 100 montgomery s...
7691,PMO,jessica claire 100 montgomery st 10th floor 55...
...,...,...
5734,Consultant,jessica claire resumesampleexamplecom 555 4321...
5191,Agriculture,kiel koelpin 6993 jacobson gardens philadelphi...
5390,Accountant,jessica claire 100 montgomery st 10th floor 55...
860,Human Resources,jessica claire 100 montgomery st 10th floor 55...


I'm also seeing duplicate entries, lets remove rows where there are duplicate resume_text data

In [5]:
# Now remove duplicates based on resume_text
df_balanced = df_balanced.drop_duplicates(subset=['resume_text']).sample(frac=1, random_state=42) 
df_balanced

# Check the new distribution
print("New Industry Distribution:")
print(df_balanced['Industry'].value_counts())
print("\nPercentages:")
print(df_balanced['Industry'].value_counts(normalize=True) * 100)

New Industry Distribution:
Industry
Information Technology       618
Software Engineering         559
Education                    388
Electrical Engineering       361
Consultant                   344
Sales                        343
Digital Media                339
Accountant                   336
Mechanical Engineer          336
Building and Construction    335
Finance                      332
Operations Manager           329
Aviation                     327
Testing                      323
Management                   319
Apparel                      315
Business Analyst             310
Public Relations             309
Civil Engineer               307
Architecture                 305
Human Resources              304
Automobile                   303
Health and Fitness           295
Banking                      292
Advocate                     280
Data Science                 274
Arts                         248
PMO                          247
Designing                    246
Agricul

In [ ]:
#from categorize_industries import categorize_industries
#categorize_industries(df_balanced)

```plaintext
Engineering = ['Mechanical Engineer', 'Electrical Engineering', 'Software Engineering', 'Civil Engineer']
Consulting = ['Consultant', 'Business Analyst', 'Management', 'PMO']
Technology = ['Information Technology', 'Data Science', 'Digital Media', 'Blockchain']
Design = ['Arts', 'Designing', 'Architecture']
Automotive = ['Automobile', 'Aviation']
Education = ['Education']
Fashion = ['Apparel']
Public Relations = ['Public Relations']
Human Resources = ['Human Resources']
Construction = ['Building and Construction']
Operations = ['Operations Manager']
Finance = ['Finance', 'Banking', 'Accountant']
Legal = ['Advocate']
Outsourcing = ['BPO']
Food and Hospitality = ['Food and Beverages']
Health and Wellness = ['Health and Fitness']
Media = ['Sales']
Science and Agriculture = ['Agriculture']
Testing = ['Testing']
```


## Code to output the class distribution of our datasets

In [12]:
# Read the CSV file
df = pd.read_csv('local_datasets/ResumeFeatures.csv')

# Get class distributions for 'Industry' and 'experience_level'
industry_distribution = df['Industry'].value_counts()
experience_level_distribution = df['experience_level'].value_counts()

# Create summary tables
industry_summary = industry_distribution.reset_index()
industry_summary.columns = ['Industry', 'Count']
industry_summary['Percentage'] = (industry_summary['Count'] / industry_summary['Count'].sum()) * 100

experience_level_summary = experience_level_distribution.reset_index()
experience_level_summary.columns = ['Experience Level', 'Count']
experience_level_summary['Percentage'] = (experience_level_summary['Count'] / experience_level_summary['Count'].sum()) * 100

# Output the summaries
print("\nIndustry Distribution Summary:")
print(industry_summary)

print("\nExperience Level Distribution Summary:")
print(experience_level_summary)


Industry Distribution Summary:
                     Industry  Count  Percentage
0      Information Technology     12    8.000000
1                       Sales     10    6.666667
2               Digital Media      9    6.000000
3            Business Analyst      8    5.333333
4        Software Engineering      8    5.333333
5          Operations Manager      8    5.333333
6                  Consultant      6    4.000000
7                Architecture      5    3.333333
8                     Banking      5    3.333333
9              Civil Engineer      5    3.333333
10            Human Resources      5    3.333333
11                  Education      5    3.333333
12  Building and Construction      5    3.333333
13                    Testing      5    3.333333
14         Health and Fitness      4    2.666667
15                    Finance      4    2.666667
16                 Accountant      4    2.666667
17     Electrical Engineering      4    2.666667
18                   Aviation      4 

In [13]:
# Read the CSV file
df = pd.read_csv('local_datasets/JobResumeMatches.csv')

# Get class distribution for 'label'
label_distribution = df['label'].value_counts()

# Create a summary table
label_summary = label_distribution.reset_index()
label_summary.columns = ['Label', 'Count']
label_summary['Percentage'] = (label_summary['Count'] / label_summary['Count'].sum()) * 100

# Output the summary
print("\nLabel Distribution Summary:")
print(label_summary)


Label Distribution Summary:
           Label  Count  Percentage
0       Good Fit     50   33.333333
1         No Fit     50   33.333333
2  Potential Fit     50   33.333333


In [14]:
# Read the CSV file
df = pd.read_csv('local_datasets/Resumes.csv')

# Get class distribution for 'label'
label_distribution = df['label'].value_counts()

# Create a summary table
label_summary = label_distribution.reset_index()
label_summary.columns = ['Label', 'Count']
label_summary['Percentage'] = (label_summary['Count'] / label_summary['Count'].sum()) * 100

# Output the summary
print("\nLabel Distribution Summary:")
print(label_summary)


Label Distribution Summary:
   Label  Count  Percentage
0      0     26   50.980392
1      1     25   49.019608


In [8]:
# df_save = df_balanced.sample(n=150, random_state=42)
# df_save['experience_level'] = None
# df_save.to_csv('local_datasets/ResumeFeatures1.csv')

In [10]:
resumes_df = ind[['resume_text']]
resumes_df = resumes_df.iloc[0:25]
resumes_df['label'] = 1
resumes_df = resumes_df.rename(columns={'resume_text': 'text'})
resumes_df.to_csv('local_datasets/Resumes.csv')

In [13]:
def get_resume_job_match_dataset(train_or_test="train"):
    """
    Returns pandas dataframe with three columns:
        1. resume_text: Has resume text
        2. job_description_text: Has job description text
        3. label: One of {No Fit, Good Fit, Potential Fit}, whether resume and job description are a fit
    """

    splits = {'train': 'train.csv', 'test': 'test.csv'}
    df = pd.read_csv("hf://datasets/cnamuangtoun/resume-job-description-fit/" + splits[train_or_test])

    return df
df = get_resume_job_match_dataset()

In [15]:
df['label'].unique()

array(['No Fit', 'Potential Fit', 'Good Fit'], dtype=object)

In [16]:
df_no_fit = df[df['label'] == 'No Fit'].sample(n=50, random_state=42)
df_potential_fit = df[df['label'] == 'Good Fit'].sample(n=50, random_state=42)
df_good_fit = df[df['label'] == 'Potential Fit'].sample(n=50, random_state=42)

sampled_df = pd.concat([df_no_fit, df_potential_fit, df_good_fit])
sampled_df = sampled_df.sample(frac=1, random_state=42).reset_index(drop=True)
sampled_df

,resume_text,job_description_text,label
0,SummaryEnergetic worker with a broad range of ...,"Reviews, analyzes, and evaluates business syst...",Good Fit
1,Professional Overview10+ years' research exper...,"Hello,Greetings from DevCare SolutionsI got an...",No Fit
2,ProfileHighly motivated Sales Associate with e...,Are you passionate about the clean tech automo...,Potential Fit
3,SummaryExperienced programmeradept at applicat...,Calling all innovators find your future at Fi...,Good Fit
4,Professional SummaryHighly motivated Sales Ass...,Industry leading client is seeking a Senior Ac...,Good Fit
...,...,...,...
145,Career OverviewA position to lead diverse team...,Job Description\nJob Title: Salesforce Communi...,Good Fit
146,ProfileActively seeking full time opportunitie...,"Were ALTEN Technology USA, an engineering comp...",Potential Fit
147,Experience10/2019to06/2019Data AnalystAngi Hom...,Requirement - Must have experience in FACETS....,No Fit
148,Professional SummaryMotivated Senior Software ...,"As a Manager, you will be a key driver in buil...",Good Fit


In [17]:
sampled_df.to_csv('local_datasets/JobResumeMatches.csv')